# Using Prosper Propensity Models (BASIC)

This notebook illustrates the use of a subset of Prosper Propensity Models based on a simple feature set called basic-geo. These models may be used for realtime inference and batch transform jobs. 

## Input Variables

**Gender** (Integer, 0-1)    
0 = Female  
1 = Male 
  
**Age Range** (Integer, 1-6)  
1 = 18-24   
2 = 25-34  
3 = 35-44  
4 = 45-54  
5 = 55-64  
6 = 65+  
  
**Household Income Range (USD)** (Integer, 0-24)  
 0 = Less than 10,000  
 1 = 10,000 to 14,999  
 2 = 15,000 to 19,999  
 3 = 20,000 to 24,999  
 4 = 25,000 to 29,999  
 5 = 30,000 to 34,999  
 6 = 35,000 to 39,999  
 7 = 40,000 to 44,999  
 8 = 45,000 to 49,999  
 9 = 50,000 to 54,999  
 10 = 55,000 to 59,999  
 11 = 60,000 to 64,999  
 12 = 65,000 to 69,999  
 13 = 70,000 to 74,999  
 14 = 75,000 to 79,999  
 15 = 80,000 to 84,999  
 16 = 85,000 to 89,999  
 17 = 90,000 to 94,999  
 18 = 95,000 to 99,999  
 19 = 100,000 to 109,999  
 20 = 110,000 to 119,999  
 21 = 120,000 to 129,999  
 22 = 130,000 to 139,999  
 23 = 140,000 to 149,999  
 24 = 150,000 or more  

## Result

The Prosper Model returns a score between 0 and 1 that represents the probability that the person has the target attribute.  


# Initialize notebook

In [14]:
import sagemaker as sm

# Specify the ARN of the model package you will be using. You can get this
# from the sagemaker console after you subscribe to the model package.
model_package_arn = 'arn goes here'



___

# Realtime Inference Example
1. Deploy model to endpoint
2. Submit inference requests to the endpoint
3. Delete the endpoint

## Deploy the model to an endpoint

In [15]:
import time

# Set endpoint name. You may replace the following with any name you like. 
endpoint_name = 'prosper-test-endpoint-' + time.strftime("%Y%m%d%H%M%S", time.localtime())

# print the critical names defined at this point
print(f'model_package_arn: {model_package_arn}')
print(f'endpoint_name: {endpoint_name}')

# Create model from model package
model = sm.ModelPackage(
            role=sm.get_execution_role(),
            model_package_arn=model_package_arn,
            sagemaker_session=sm.Session())

# Deploy the model to an endpoint. Be sure to delete the endpoint when you are finished with it.
# By default, this method waits until the endpoint is deployed. This could take a while.
# To have the API return immediately, set the wait parameter to false. Note, however, that you
# cannot submit a request to the endpoint until it is in service. The easy way to check the status is
# by using the sagemaker console.
model.deploy(1, 'ml.m4.xlarge', endpoint_name=endpoint_name)

print(f'\nEndpoint {endpoint_name} is now in service')

model_package_arn: arn:aws:sagemaker:us-east-2:214666064132:model-package/trends-styles-important-basic
endpoint_name: prosper-test-endpoint-20191124202027
-------------------------------------------------------------------------------------!
Endpoint prosper-test-endpoint-20191124202027 is now in service


## Submit a realtime inference request to the endpoint created above

In [16]:
# create predictor for endpoint created above
predictor = sm.predictor.RealTimePredictor(endpoint_name, content_type='text/csv')

# Sample request
# Female
gender = 0
# Age Range: 18-24
age_range = 1
# Household Income: $75,000 to $79,999
household_income_range = 14

# format request data as comma-delimited string
request_data = f'{gender},{age_range},{household_income_range}'
print(f'Request Data: {request_data}')

# Submit the request to the endpoint.
# By default, the result is returned as a sequence of bytes. We decode it as utf-8 string.
# Note that there are parameters available for serializing and deserializing input and output data.
result = predictor.predict(request_data).decode('utf-8')

# print the result. This is the probability that the person with the requested parameters are in the
# target class.
print(f'Response (probability): {result}')

Request Data: 0,1,14
Response (probability): 0.7214754223823547


## Delete the endpoint

In [17]:
sm.session.Session().delete_endpoint(endpoint_name)
sm.session.Session().delete_endpoint_config(endpoint_name)

  
***
    
# Batch Transform Example
1. Prepare the input file
2. Create the transform job

## Prepare the input file
Upload the sample input file to S3

In [12]:
# A sample input file (prosper_data/basic_test_data.csv) is provided with this notebook.
# The transform job expects its input file to to live in S3.  We upload the file
# to the default bucket with key_prefix of prosper-sample-data. You
# can upload to any bucket and key you like if you specify the bucket and key_prefix parameters.

transform_input = sm.Session().upload_data('prosper_data/basic_test_data.csv', key_prefix='prosper-sample-data')
print(f'transform input: {transform_input}')

transform input: s3://sagemaker-us-east-2-214666064132/prosper-sample-data/basic_test_data.csv


## Create the transform job

In [13]:
# Create model from model package
model = sm.ModelPackage(
            role=sm.get_execution_role(),
            model_package_arn=model_package_arn,
            sagemaker_session=sm.Session())

# Create the transformer
# A variety of parameters may be specified here including the output path where
# SageMaker will send the results of the transform. Since we do not specify the output,
# Sagemaker will leave the results in the default bucket. We will retrieve this location 
# below so that we can inspect the output.
transformer = model.transformer(1, 'ml.m4.xlarge')

# Run the transform job.
# By default, the output file contains only the inference result for each row.
# You can use the output_filter parameter to include any of the input columns. Review also
# input_filter which allows you to filter the parameters passed as input to the transformer.
# The combination of input_filter and output_filter gives you a lot of flexibility.
# By default, the API does not wait for the transform job to complete. You can control this with
# the wait parameter.
transformer.transform(transform_input, content_type='text/csv')

# The transform job sets the output path in the output_path member.
print(f'Transform output: {transformer.output_path}')

Transform output: s3://sagemaker-us-east-2-214666064132/trends-styles-important-basic-2019-11-2-2019-11-24-17-19-47-932
